In [1]:
import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tensorflow.contrib.learn.python.learn.datasets.mnist import extract_images, extract_labels

In [2]:
class AI(nn.Module):
    def __init__(self):
        super(AI, self).__init__()
        self.lstm_layers = 1
        self.lstm_depth = 1000
        self.cnn1 = nn.Conv2d(1,128,5,stride=2,padding=1)
        self.cnn2 = nn.Conv2d(128,128,3,stride=2,padding=0)
        self.rnn = nn.LSTM(6 * 6, self.lstm_depth,self.lstm_layers)
        self.fc1 = nn.Linear(self.lstm_depth,self.lstm_depth)
        self.fc2 = nn.Linear(self.lstm_depth,10)
        self.out = nn.Dropout(0.9)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')     
        self.to(self.device)
    def forward(self,x):
        hx = T.zeros(self.lstm_layers,128, self.lstm_depth).to(self.device)
        cx = T.zeros(self.lstm_layers,128, self.lstm_depth).to(self.device)
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn2(x))
#        print(x.shape)
        x = x.view(-1,128,6 * 6)
        hx, cx = self.rnn(x, (hx, cx))
        o = T.tanh(self.fc1(hx))
        o = T.sigmoid(self.fc2(o))
#        o = self.out(o)
        return o

In [3]:
class PLAYFIELD():
    def __init__(self,width,height):
        self.width = width
        self.height = height
        self.pf = np.zeros((self.width,self.height,2))
    def setStone(self,x,y,stone):
        s = np.zeros(2)
        s[stone] = 1
        self.pf[y][x] = s
    def get(self):
        return self.pf

In [4]:
with open('train-images-idx3-ubyte.gz', 'rb') as f:
    train_images = extract_images(f)
with open('train-labels-idx1-ubyte.gz', 'rb') as f:
    train_labels = extract_labels(f)

with open('t10k-images-idx3-ubyte.gz', 'rb') as f:
    test_images = extract_images(f)
with open('t10k-labels-idx1-ubyte.gz', 'rb') as f:
    test_labels = extract_labels(f)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-labels-idx1-ubyte.gz
Extracting t10k-images-idx3-ubyte.gz
Extracting t10k-labels-idx1-ubyte.gz


In [5]:


ai = AI()

max_mem = 1000
a = 28

inp = []
for i in range(max_mem):
    inp.append(np.random.randn(1, a, 28))
    
criterion = nn.MSELoss()#SmoothL1Loss()MSELoss
optimizer = optim.RMSprop(ai.parameters(), lr=0.00005)

losses = 0
for j in range(1):
    for i in range(60000):#train_images.shape[0]):
        def closure():
            optimizer.zero_grad()
            inp1 = train_images[i]#inp[i % max_mem].copy() + np.random.randn(1, a, 28) * 0.1
            inp1.shape = (1,1,28,28)
            inp1 = T.from_numpy(inp1).float().to(ai.device)
            out = ai(inp1)
    #        print(out)
    #        print(target)
            target = out.cpu().detach().numpy().copy()
    #        print(target)
            target[0][5 - 1] = np.zeros(10)
            target[0][5 - 1][train_labels[i]] = 1.0
    #        print(target)
            target = T.from_numpy(np.array(target)).to(ai.device)
            loss = criterion(out, target.float())
            loss.backward()
            return loss
        loss = optimizer.step(closure)
        losses = losses + loss.item()
        if (i % 100) == 0:
            print('Step: ' + str(i) + ' loss: ' + str(losses / 100))
            losses = 0

Step: 0 loss: 1.9638673402369024e-05
Step: 100 loss: 0.0007004756139940582
Step: 200 loss: 0.0005376565999176819
Step: 300 loss: 0.0004265303796273656
Step: 400 loss: 0.0003312422920498648
Step: 500 loss: 0.0003355038977315417
Step: 600 loss: 0.00034286423000594367
Step: 700 loss: 0.00030643920774309664
Step: 800 loss: 0.00026470941334991947
Step: 900 loss: 0.0002709677997881954
Step: 1000 loss: 0.00022747866154531948
Step: 1100 loss: 0.0002688650946856797
Step: 1200 loss: 0.00022410556820432248
Step: 1300 loss: 0.0002420017381245998
Step: 1400 loss: 0.000204765021435378
Step: 1500 loss: 0.00018194442011918
Step: 1600 loss: 0.00018101563234267814
Step: 1700 loss: 0.00014436505884987127
Step: 1800 loss: 0.0001170555394674011
Step: 1900 loss: 0.00016016175487948203
Step: 2000 loss: 0.0001536472590669291
Step: 2100 loss: 0.00015528825632827648
Step: 2200 loss: 0.00011827979105873964
Step: 2300 loss: 0.00012412154587991607
Step: 2400 loss: 0.00012199567477637174
Step: 2500 loss: 0.00014905

Step: 20700 loss: 9.52728726983132e-05
Step: 20800 loss: 0.00012032320454645529
Step: 20900 loss: 7.165671039063959e-05
Step: 21000 loss: 0.0001048702426290049
Step: 21100 loss: 6.0942875899110404e-05
Step: 21200 loss: 5.8998262617793173e-05
Step: 21300 loss: 6.0858318921743756e-05
Step: 21400 loss: 7.077841009170527e-05
Step: 21500 loss: 6.791051914708745e-05
Step: 21600 loss: 5.177772389643564e-05
Step: 21700 loss: 5.760674571440916e-05
Step: 21800 loss: 2.3645362372857193e-05
Step: 21900 loss: 2.1310883492811073e-05
Step: 22000 loss: 7.8488156458274e-05
Step: 22100 loss: 3.7347741937256986e-05
Step: 22200 loss: 0.00012649852589868615
Step: 22300 loss: 8.976927567952764e-05
Step: 22400 loss: 2.177084785008887e-05
Step: 22500 loss: 0.0001231174717775385
Step: 22600 loss: 0.0001196998423952067
Step: 22700 loss: 8.935853692513263e-05
Step: 22800 loss: 8.135812164420387e-05
Step: 22900 loss: 7.201336947272096e-05
Step: 23000 loss: 1.3525111050016747e-05
Step: 23100 loss: 4.85165651118801

Step: 41100 loss: 7.802137523651497e-05
Step: 41200 loss: 2.4087464209276987e-05
Step: 41300 loss: 8.366986054001102e-05
Step: 41400 loss: 7.611611990824496e-05
Step: 41500 loss: 7.245426083707596e-05
Step: 41600 loss: 6.632443890121742e-05
Step: 41700 loss: 4.1100504417765914e-05
Step: 41800 loss: 5.56182418838036e-05
Step: 41900 loss: 2.607720473967845e-05
Step: 42000 loss: 8.020968543913876e-05
Step: 42100 loss: 5.0018822789492654e-05
Step: 42200 loss: 4.818659567442207e-05
Step: 42300 loss: 2.3966166835425185e-05
Step: 42400 loss: 6.731133493913855e-05
Step: 42500 loss: 0.00010733863964531243
Step: 42600 loss: 3.984957869584527e-05
Step: 42700 loss: 5.521971187022245e-05
Step: 42800 loss: 3.729451919753712e-05
Step: 42900 loss: 6.504462399719247e-05
Step: 43000 loss: 7.629647258866056e-05
Step: 43100 loss: 7.39712824751182e-05
Step: 43200 loss: 5.679566830710025e-05
Step: 43300 loss: 6.289838543783899e-05
Step: 43400 loss: 1.4398970742892647e-05
Step: 43500 loss: 4.6529064986700064

In [6]:
count = 0
for i in range(10000):#train_images.shape[0]):
    optimizer.zero_grad()
    inp1 = test_images[i]#inp[i % max_mem].copy()
    inp1.shape = (1,1,28,28)
    inp1 = T.from_numpy(inp1).float().to(ai.device)
    out = ai(inp1).cpu().detach().numpy()
    m = np.argmax(out[0][5 - 1])
    if m == test_labels[i]:
#        print(m)
        count = count + 1
print("correct: " + str(count / 10000))#train_images.shape[0]))

correct: 0.9593
